## 🧠 DINOv2 Multi-Task Pipeline

This notebook implements a **multi-task architecture** based on **DINOv2** —  
a frozen vision backbone that powers two lightweight heads:  
**Segmentation** (detecting manipulated regions) and  
**Classification** (authentic vs forged).

---

### ⚙️ Architecture Overview
- **Encoder** → DINOv2-Base (frozen feature extractor)  
- **Segmentation Head** → compact Conv–ReLU–Conv decoder  
- **Classification Head** → linear layer over the CLS token  
- **Optimization** → `AdamW` + `BCEWithLogitsLoss` / `CrossEntropyLoss`  
- **Metrics** → IoU, Dice, Pixel Accuracy  

---

### 🔄 Pipeline Steps
1. **Dataset Loading** → authentic / forged images + mask files (`.npy`)  
2. **Feature Extraction** → DINOv2 encoder  
3. **Segmentation** → binary mask prediction of forged regions  
4. **Classification** → authenticity decision (A / F)  
5. **Evaluation** → IoU, Dice & Accuracy on validation set  

---

> 🧩 **Note:** The model runs in **Offline Kaggle** mode —  
> DINOv2 is loaded from `/kaggle/input/dinov2/pytorch/base/1`.


In [ ]:
import os, gc, math, random, json
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm

# ----------------------------
# Prepare datasets and loaders
# ----------------------------
train_cls_dataset = ForgeryClsDataset(train_auth, train_forg, IMG_SIZE)
val_cls_dataset = ForgeryClsDataset(val_auth, val_forg, IMG_SIZE)

train_cls_loader = DataLoader(
    train_cls_dataset, batch_size=BATCH_CLS, shuffle=True, num_workers=2, pin_memory=True
)
val_cls_loader = DataLoader(
    val_cls_dataset, batch_size=BATCH_CLS, shuffle=False, num_workers=2, pin_memory=True
)

# ----------------------------
# SEGMENTATION TRAINING
# ----------------------------
seg_criterion = nn.BCEWithLogitsLoss()
seg_optimizer = optim.AdamW(model.seg_head.parameters(), lr=LR_SEG, weight_decay=WEIGHT_DECAY)

for epoch in range(1, EPOCHS_SEG + 1):
    model.train()
    train_loss = 0.0

    for imgs, masks in tqdm(train_seg_loader, desc=f"[Seg] Epoch {epoch}/{EPOCHS_SEG}"):
        imgs, masks = imgs.to(device), masks.to(device)

        logits = model.forward_seg(imgs)
        loss = seg_criterion(logits, masks)

        seg_optimizer.zero_grad()
        loss.backward()
        seg_optimizer.step()

        train_loss += loss.item() * imgs.size(0)

    train_loss /= len(train_seg_loader.dataset)

    # ----------------------------
    # Validation
    # ----------------------------
    model.eval()
    val_loss, total_iou, total_dice, total_acc = 0.0, 0.0, 0.0, 0.0

    with torch.no_grad():
        for imgs, masks in val_seg_loader:
            imgs, masks = imgs.to(device), masks.to(device)
            logits = model.forward_seg(imgs)
            loss = seg_criterion(logits, masks)
            val_loss += loss.item() * imgs.size(0)

            probs = torch.sigmoid(logits).cpu().numpy()
            gts = masks.cpu().numpy()

            for p, g in zip(probs, gts):
                p = p[0]
                g = g[0]
                total_iou += iou_score(p, g)
                total_dice += dice_score(p, g)
                total_acc += pixel_acc(p, g)

    n_val = len(val_seg_loader.dataset)
    print(
        f"[Seg] Epoch {epoch} | train_loss={train_loss:.4f} | val_loss={val_loss/n_val:.4f} "
        f"| IoU={total_iou/n_val:.3f} | Dice={total_dice/n_val:.3f} | Acc={total_acc/n_val:.3f}"
    )

    torch.cuda.empty_cache()
    gc.collect()

# ----------------------------
# CLASSIFICATION TRAINING
# ----------------------------
cls_criterion = nn.CrossEntropyLoss()
cls_optimizer = optim.AdamW(model.cls_head.parameters(), lr=LR_CLS, weight_decay=WEIGHT_DECAY)

for epoch in range(1, EPOCHS_CLS + 1):
    model.train()
    train_loss = 0.0

    for imgs, labels in tqdm(train_cls_loader, desc=f"[Cls] Epoch {epoch}/{EPOCHS_CLS}"):
        imgs, labels = imgs.to(device), labels.to(device)

        logits = model.forward_cls(imgs)
        loss = cls_criterion(logits, labels)

        cls_optimizer.zero_grad()
        loss.backward()
        cls_optimizer.step()

        train_loss += loss.item() * imgs.size(0)

    train_loss /= len(train_cls_loader.dataset)

    # ----------------------------
    # Validation
    # ----------------------------
    model.eval()
    correct, total = 0, 0

    with torch.no_grad():
        for imgs, labels in val_cls_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            logits = model.forward_cls(imgs)
            preds = logits.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    print(f"[Cls] Epoch {epoch} | train_loss={train_loss:.4f} | val_acc={100.0*correct/total:.2f}%")

    torch.cuda.empty_cache()
    gc.collect()



🚀 Inference & Submission Phase

In [ ]:
def predict_mask_prob(img_pil):
    """Predict mask probability for a single PIL image"""
    img = img_pil.convert("RGB").resize((IMG_SIZE, IMG_SIZE), Image.BILINEAR)
    x = torch.from_numpy(np.array(img, dtype=np.float32)/255.).permute(2,0,1)[None].to(device)
    with torch.no_grad():
        logits = model.forward_seg(x)
        prob = torch.sigmoid(logits)[0,0].cpu().numpy()
    return prob


In [ ]:
def tta_predict(img_pil):
    """Horizontal flip TTA prediction"""
    prob = predict_mask_prob(img_pil)
    img_flip = img_pil.transpose(Image.FLIP_LEFT_RIGHT)
    prob_flip = predict_mask_prob(img_flip)
    prob = 0.5 * (prob + prob_flip[:, ::-1])  # average TTA
    return prob


In [ ]:
from skimage import measure

def post_process(mask, min_size=20):
    """Remove small blobs to reduce false positives"""
    labeled = measure.label(mask)
    new_mask = np.zeros_like(mask, dtype=np.uint8)
    for region in measure.regionprops(labeled):
        if region.area >= min_size:
            coords = region.coords
            new_mask[coords[:,0], coords[:,1]] = 1
    return new_mask


In [ ]:
def rle_encode_numpy(mask):
    """Mask (2D numpy) to RLE list"""
    pixels = mask.flatten(order='F')
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return runs.tolist()


In [ ]:
def binarize_mask(prob, thr=0.5):
    """Convert probability mask to binary mask"""
    return (prob > thr).astype(np.uint8)


In [ ]:
def write_submission(rows, sample_csv="sample_submission.csv", out_csv="submission.csv"):
    sub = pd.DataFrame(rows, columns=["case_id","annotation"])
    if os.path.exists(sample_csv):
        ss = pd.read_csv(sample_csv)
        ss["case_id"] = ss["case_id"].astype(str)
        if not sub.empty:
            sub["case_id"] = sub["case_id"].astype(str)
            sub = ss[["case_id"]].merge(sub, on="case_id", how="left")
            sub["annotation"] = sub["annotation"].fillna("authentic")
        else:
            sub = ss[["case_id"]].copy()
            sub["annotation"] = "authentic"
    sub.to_csv(out_csv, index=False)
    print("Wrote submission:", out_csv)


In [ ]:

# Inference + Submission (segmentation -> RLE, sinon 'authentic')

def predict_mask_prob(img_pil):
    img = img_pil.convert("RGB").resize((IMG_SIZE, IMG_SIZE), Image.BILINEAR)
    x = torch.from_numpy(np.array(img, dtype=np.float32)/255.).permute(2,0,1)[None].to(device)
    with torch.no_grad():
        logits = model.forward_seg(x)
        prob = torch.sigmoid(logits)[0,0].cpu().numpy()
    return prob

THR = 0.5
rows = []
if os.path.exists(TEST_DIR):
    test_files = sorted(os.listdir(TEST_DIR))
    print("Test images:", len(test_files))
    for fname in tqdm(test_files, desc="Inference"):
        case_id = Path(fname).stem
        path = str(Path(TEST_DIR)/fname)
        pil = Image.open(path).convert("RGB")
        ow, oh = pil.size

        prob = predict_mask_prob(pil)  # IMG_SIZE x IMG_SIZE
        # resize to original size
        mask = cv2.resize(prob, (ow, oh), interpolation=cv2.INTER_NEAREST)
        binm = (mask > THR).astype(np.uint8)

        if binm.sum() == 0:
            annot = "authentic"
        else:
            annot = rle_encode_numpy(binm)  # JSON list of [start,length,...]

        rows.append({"case_id": case_id, "annotation": annot})

sub = pd.DataFrame(rows, columns=["case_id","annotation"])

# Aligne avec sample_submission
if os.path.exists(SAMPLE_SUB):
    ss = pd.read_csv(SAMPLE_SUB)
    ss["case_id"] = ss["case_id"].astype(str)
    if not sub.empty:
        sub["case_id"] = sub["case_id"].astype(str)
        sub = ss[["case_id"]].merge(sub, on="case_id", how="left")
        sub["annotation"] = sub["annotation"].fillna("authentic")
    else:
        sub = ss[["case_id"]].copy()
        sub["annotation"] = "authentic"

# Sauvegarde
OUT_PATH = "submission.csv"
sub.to_csv(OUT_PATH, index=False)
print(" Wrote submission:", OUT_PATH)
print(sub.head())


🖼️ Forgery Mask Visualization



In [ ]:
import matplotlib.pyplot as plt


# Function: predict probability mask

def predict_mask_prob(img_pil):
    """Predict a pixel-wise probability mask for a given image."""
    img = img_pil.convert("RGB").resize((IMG_SIZE, IMG_SIZE), Image.BILINEAR)
    x = torch.from_numpy(np.array(img, dtype=np.float32) / 255.).permute(2, 0, 1)[None].to(device)
    with torch.no_grad():
        logits = model.forward_seg(x)
        prob = torch.sigmoid(logits)[0, 0].cpu().numpy()
    return prob

# Visualization on random test images

N_SHOW = 5  # number of test images to visualize
THR = 0.5   # threshold for binary mask

if os.path.exists(TEST_DIR):
    test_files = sorted(os.listdir(TEST_DIR))
    samples = random.sample(test_files, min(N_SHOW, len(test_files)))

    plt.figure(figsize=(14, N_SHOW * 4))
    for i, fname in enumerate(samples, 1):
        path = str(Path(TEST_DIR) / fname)
        pil = Image.open(path).convert("RGB")
        ow, oh = pil.size

        # Predict probability mask
        prob = predict_mask_prob(pil)
        mask_resized = cv2.resize(prob, (ow, oh), interpolation=cv2.INTER_NEAREST)
        binary_mask = (mask_resized > THR).astype(np.uint8)

        # Visualization: original / prob mask / overlay
        plt.subplot(N_SHOW, 3, 3*(i-1)+1)
        plt.imshow(pil)
        plt.title(f"Original - {fname}")
        plt.axis("off")

        plt.subplot(N_SHOW, 3, 3*(i-1)+2)
        plt.imshow(mask_resized, cmap="viridis")
        plt.title("Probability Mask")
        plt.axis("off")

        plt.subplot(N_SHOW, 3, 3*(i-1)+3)
        plt.imshow(pil)
        plt.imshow(binary_mask, cmap="Reds", alpha=0.4)
        plt.title("Overlay Mask")
        plt.axis("off")

    plt.tight_layout()
    plt.show()



🧩 Authentic Image Mask Visualization



In [ ]:
import matplotlib.pyplot as plt

# Function: predict mask on tensor batch

def predict_val_batch(model, loader, n_show=5, thr=0.5):
    """Display a few validation images with GT mask and predicted mask."""
    model.eval()
    shown = 0

    plt.figure(figsize=(12, n_show * 4))

    with torch.no_grad():
        for imgs, masks in loader:
            imgs, masks = imgs.to(device), masks.to(device)
            logits = model.forward_seg(imgs)
            probs = torch.sigmoid(logits).cpu().numpy()
            imgs_np = imgs.cpu().permute(0, 2, 3, 1).numpy()
            gts = masks.cpu().numpy()

            for j in range(len(imgs)):
                if shown >= n_show:
                    break

                # Retrieve elements
                img = np.clip(imgs_np[j], 0, 1)
                gt = gts[j, 0]
                prob = probs[j, 0]
                pred_bin = (prob > thr).astype(np.uint8)

                # Show three panels
                plt.subplot(n_show, 3, 3 * shown + 1)
                plt.imshow(img)
                plt.title("Original")
                plt.axis("off")

                plt.subplot(n_show, 3, 3 * shown + 2)
                plt.imshow(gt, cmap="gray")
                plt.title("Ground Truth")
                plt.axis("off")

                plt.subplot(n_show, 3, 3 * shown + 3)
                plt.imshow(img)
                plt.imshow(pred_bin, cmap="Reds", alpha=0.4)
                plt.title("Predicted Mask")
                plt.axis("off")

                shown += 1

            if shown >= n_show:
                break

    plt.tight_layout()
    plt.show()

# Run visualization
predict_val_batch(model, val_seg_loader, n_show=5, thr=0.5)



🔥 Forged Image Mask Visualization



In [ ]:
import matplotlib.pyplot as plt

def show_forged_examples(model, loader, n_show=5, thr=0.5):
    """Show only forged examples from validation set (mask not empty)."""
    model.eval()
    shown = 0

    plt.figure(figsize=(12, n_show * 4))

    with torch.no_grad():
        for imgs, masks in loader:
            imgs, masks = imgs.to(device), masks.to(device)
            logits = model.forward_seg(imgs)
            probs = torch.sigmoid(logits).cpu().numpy()
            imgs_np = imgs.cpu().permute(0, 2, 3, 1).numpy()
            gts = masks.cpu().numpy()

            for j in range(len(imgs)):
                gt = gts[j, 0]
                if gt.sum() == 0:  # skip authentic images
                    continue

                img = np.clip(imgs_np[j], 0, 1)
                prob = probs[j, 0]
                pred_bin = (prob > thr).astype(np.uint8)

                # 3 panels: original / GT / predicted
                plt.subplot(n_show, 3, 3 * shown + 1)
                plt.imshow(img)
                plt.title("Original (Forged)")
                plt.axis("off")

                plt.subplot(n_show, 3, 3 * shown + 2)
                plt.imshow(gt, cmap="gray")
                plt.title("Ground Truth Mask")
                plt.axis("off")

                plt.subplot(n_show, 3, 3 * shown + 3)
                plt.imshow(img)
                plt.imshow(pred_bin, cmap="Reds", alpha=0.4)
                plt.title("Predicted Mask")
                plt.axis("off")

                shown += 1
                if shown >= n_show:
                    break

            if shown >= n_show:
                break

    plt.tight_layout()
    plt.show()

#  Run visualization
show_forged_examples(model, val_seg_loader, n_show=5, thr=0.5)
